Goal: separate the ingrdients in the FoodKG table into a list of ingredients

In [1]:
import json
import os

import pandas as pd
from dotenv import load_dotenv

from llm_util import process_branded_food_experimental_df, assemble_branded_food_experimental_df

load_dotenv()  # Take environment variables from .env
api_key = os.getenv("OPENAI_API_KEY")

from openai import OpenAI

client = OpenAI(
    base_url="http://localhost:1234/v1",  # local model
    api_key=api_key
)

In [2]:
df = pd.read_csv("data/raw/usda_2022_food_branded_experimental_DESCRIPTION_ONLY.csv")

In [3]:
df.head(5)

,fdc_id,description
0,167512,"Pillsbury Golden Layer Buttermilk Biscuits, Ar..."
1,167513,"Pillsbury, Cinnamon Rolls with Icing, refriger..."
2,167514,"Kraft Foods, Shake N Bake Original Recipe, Coa..."
3,167515,"George Weston Bakeries, Thomas English Muffins"
4,167516,"Waffles, buttermilk, frozen, ready-to-heat"


In [5]:
model = "qwen/qwen3-4b-2507"
# model="openai/gpt-oss-20b",
process_branded_food_experimental_df(
    df=df,
    model="qwen/qwen3-4b-2507",
    client=client,
    restart=True,
    batch_size=100,
    stop_at=1000,
)

BadRequestError: Error code: 400 - {'error': 'The number of tokens to keep from the initial prompt is greater than the context length. Try to load the model with a larger context length, or provide a shorter input'}

In [ ]:
final_df = assemble_branded_food_experimental_df()

In [ ]:
len(final_df)

In [ ]:
# CSV can’t store native Python lists (they are objects), so use JSON dump
product_table = pd.DataFrame({
    "fdc_id": final_df["fdc_id"],
    "description": final_df["description"],
    "mapped_ingredient": final_df["mapped_ingredient"].apply(json.dumps),
}).sort_values("fdc_id")

In [ ]:
product_table.head(5)

In [ ]:
# Create directory if it doesn't exist and save
os.makedirs("data/output", exist_ok=True)
product_table.to_csv("data/output/processed_products.csv", index=False)